# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [2]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from pandasql import sqldf

In [3]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [4]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [5]:
# Create a Beautiful Soup Object
html = browser.html

soup = BeautifulSoup(html, "html.parser")

In [6]:
# Extract all rows of data
all_data = soup.find_all(class_='data-row')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [7]:
# Create an empty list
data_list = []
# Loop through the scraped data to create a list of rows
for data in all_data:
    #getting all values in rows
    raw_values = data.find_all('td')
    
    #list comprehension
    values = [data.text.strip() for data in raw_values]
    
    data_dict = {
        "ID" : values[0],
        "terrestrial_date" : values[1],
        "sol" : values[2],
        "ls" : values[3],
        "month" : values[4],
        "min_temp" : values[5],
        "pressure" : values[6]   
    }
    
    data_list.append(data_dict)

In [8]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(data_list)

In [9]:
# Confirm DataFrame was created successfully
df.head()

,ID,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [10]:
# Examine data type of each column
df.dtypes

ID                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object

In [11]:
# Change data types for data analysis
df['ID'] = df["ID"].astype(int)
df["terrestrial_date"] = df["terrestrial_date"].astype('datetime64[ns]')
df['sol'] = df["sol"].astype(int)
df['ls'] = df['ls'].astype(int)
df["month"] = df["month"].astype(int)
df['min_temp'] = df['min_temp'].astype(float)
df['pressure'] = df['pressure'].astype(float)


In [13]:
# Confirm type changes were successful by examining data types again
df.dtypes

ID                           int32
terrestrial_date    datetime64[ns]
sol                          int32
ls                           int32
month                        int32
min_temp                   float64
pressure                   float64
dtype: object

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [14]:
# 1. How many months are there on Mars?
months_onmars = df['month'].max()
months_onmars

12

In [15]:
# 2. How many Martian days' worth of data are there
date_query = """ 
            SELECT COUNT(terrestrial_date) AS Martian_days 
            FROM df
            """
result = sqldf(date_query)

print(result)

   Martian_days
0          1867


In [16]:
# 3. What is the average low temperature by month?
monthly_average = """
                SELECT month, AVG(min_temp) AS Average_low_temp
                FROM df 
                GROUP BY month
                ORDER BY month
                """

result = sqldf(monthly_average)

print(result)

    month  Average_low_temp
0       1        -77.160920
1       2        -79.932584
2       3        -83.307292
3       4        -82.747423
4       5        -79.308725
5       6        -75.299320
6       7        -72.281690
7       8        -68.382979
8       9        -69.171642
9      10        -71.982143
10     11        -71.985507
11     12        -74.451807


In [17]:
# Plot the average temperature by month


In [18]:
# Identify the coldest and hottest months in Curiosity's location



In [19]:
# 4. Average pressure by Martian month
average_pressure = """
                SELECT month, AVG(pressure) AS Average_pressure
                FROM df 
                GROUP BY month
                ORDER BY month
                """

result = sqldf(average_pressure)

print(result)

    month  Average_pressure
0       1        862.488506
1       2        889.455056
2       3        877.322917
3       4        806.329897
4       5        748.557047
5       6        745.054422
6       7        795.105634
7       8        873.829787
8       9        913.305970
9      10        887.312500
10     11        857.014493
11     12        842.156627


In [20]:
# Plot the average pressure by month


In [21]:
# 5. How many terrestrial (earth) days are there in a Martian year?


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [18]:
# Write the data to a CSV
df.to_csv("Output/Mars.csv")

In [23]:
browser.quit()